In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [ ]:
train_data = pd.read_csv('./data/splited_train.csv')
test_data = pd.read_csv('./data/splited_test.csv')


In [ ]:
train_input = list(train_data['question_text'])
train_label = list(train_data['target'])

test_input = list(test_data['question_text'])

In [ ]:

#remove all the stop words for the 
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop=stopwords.words('english') 

def remove_stop_words(x):
    for word in stop:
        token = " " + word + " "
        if (x.find(token) != -1):
            x = x.replace(token, " ")
    return x

# train_input_rsw = list(map(remove_stop_words, train_input))
# test_input_rsw = list(map(remove_stop_words, test_input))
train_input_rsw=train_input
test_input_rsw=test_input

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
max_features=95000
embed_size = 300 
max_length = 60 

In [ ]:
def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')

embeddings_index = {}
with open("./data/embeddings/glove.840B.300d/glove.840B.300d.txt",'r') as f:
    for line in f:
        
        word,coefs=get_coefs(*line.split(" "))
        #coefs = np.asarray(coefs, dtype='float32')
        embeddings_index[word] = coefs

In [ ]:
tokenizer=Tokenizer(num_words=max_features)

tokenizer.fit_on_texts(train_input_rsw)



word_index = tokenizer.word_index
n_words=min(max_features,len(word_index))

# embedding_matrix = np.zeros((n_words+1, embed_size))

# for word, i in word_index.items():
#     if i >= max_features: 
#         continue
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None: 
#         embedding_matrix[i] = embedding_vector
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


In [ ]:
sequences = tokenizer.texts_to_sequences(train_input_rsw)

train_input_padded = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')
print(train_input_padded.shape)

sequences = tokenizer.texts_to_sequences(test_input_rsw)
test_input_padded = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')
print(test_input_padded.shape)

(1044897, 60)
(261225, 60)


In [ ]:
train_text, cv_text, train_target, cv_target = train_test_split(train_input_padded, train_label, test_size = 0.1, random_state=2)

In [ ]:
# https://www.kaggle.com/qqgeogor/keras-lstm-attention-glove840b-lb-0-043
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = self.add_weight(shape=(input_shape[-1],),
                                 name='{}_W'.format(self.name),
                                 initializer=self.init,
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[1],),
                                     name='{}_b'.format(self.name),
                                     initializer='zero',
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None
        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [ ]:
from keras.layers import Embedding,Bidirectional,LSTM,Dropout,Conv1D,MaxPooling1D,Dense
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D
from keras.models import Model
from keras.optimizers import Adam

In [ ]:
inp = Input(shape=(max_length,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Bidirectional(LSTM(64, return_sequences=True))(x)
x=Dropout(0.4)(x)
x = Attention(max_length)(x)
x = Dense(64, activation="relu")(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=1e-3), metrics=['acc'])

In [ ]:
# del embeddings_index
# import gc
# gc.collect()

In [ ]:
history=model.fit(np.array(train_text), np.array(train_target), epochs =5, validation_data=(np.array(cv_text),np.array(cv_target)), batch_size=1024,verbose=1)

Epoch 1/5
919/919 [==============================] - 111s 96ms/step - loss: 0.1467 - acc: 0.9458 - val_loss: 0.1104 - val_acc: 0.9562
Epoch 2/5
919/919 [==============================] - 87s 94ms/step - loss: 0.1074 - acc: 0.9573 - val_loss: 0.1047 - val_acc: 0.9584
Epoch 3/5
919/919 [==============================] - 87s 94ms/step - loss: 0.1007 - acc: 0.9599 - val_loss: 0.1021 - val_acc: 0.9594
Epoch 4/5
919/919 [==============================] - 86s 94ms/step - loss: 0.0952 - acc: 0.9619 - val_loss: 0.1011 - val_acc: 0.9599
Epoch 5/5
919/919 [==============================] - 86s 94ms/step - loss: 0.0907 - acc: 0.9634 - val_loss: 0.0996 - val_acc: 0.9606


In [ ]:
from sklearn.metrics import f1_score
cv_predictions = model.predict(cv_text, batch_size=512)

thresholds = []
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    result = f1_score(cv_target, (cv_predictions>thresh).astype(int))
    thresholds.append([thresh, result])
    print("F1 score at threshold {} is {}".format(thresh, result))

thresholds.sort(key=lambda x: x[1], reverse=True)
print("Best value {}".format(thresholds[0]))
best_thresh = thresholds[0]

F1 score at threshold 0.1 is 0.6206101843722563
F1 score at threshold 0.11 is 0.6273128738330818
F1 score at threshold 0.12 is 0.6340100364963503
F1 score at threshold 0.13 is 0.6377113736391012
F1 score at threshold 0.14 is 0.6441833137485311
F1 score at threshold 0.15 is 0.6486486486486487
F1 score at threshold 0.16 is 0.6531798909751665
F1 score at threshold 0.17 is 0.6567916410571604
F1 score at threshold 0.18 is 0.659155981575999
F1 score at threshold 0.19 is 0.6624653390471388
F1 score at threshold 0.2 is 0.665561224489796
F1 score at threshold 0.21 is 0.6678719008264462
F1 score at threshold 0.22 is 0.6696673189823875
F1 score at threshold 0.23 is 0.6693936794880253
F1 score at threshold 0.24 is 0.6714181043095984
F1 score at threshold 0.25 is 0.6723708145253655
F1 score at threshold 0.26 is 0.6733818825290956
F1 score at threshold 0.27 is 0.6738070717473394
F1 score at threshold 0.28 is 0.675949016348019
F1 score at threshold 0.29 is 0.677705582337735
F1 score at threshold 0.3 

In [ ]:
#run on test set with best threshold
total_predictions=model.predict(test_input_padded, batch_size=512)
test_labels=list(test_data['target'])
predictions1 = (total_predictions>best_thresh).astype(int)
res=f1_score(test_labels, predictions1[:,0])
print(res)

0.6751319069983185
